## Authentification to Google Earth Engine

In [13]:
import os
import json

import geopandas as gpd
import datetime

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g5aM6YhEAsT-P7Asnq8dGzpWbnySCCQh46zWdmRfYbebrvOGoEPfhg

Successfully saved authorization token.


## Area of interest (AOI) to GEE

In [3]:
aoi_path = 'data/aoi.shp'

In [20]:
# AOI to EE

gpd_aoi = gpd.read_file(aoi_path)
gjson = gpd_aoi.to_crs(epsg='4326').to_json()


def geojson_to_ee(geo_json: str, geodesic=True) -> ee.Geometry:
    """Converts a geojson to ee.Geometry()
    Args:
        geo_json: geojson definition as str.
    Returns:
        ee_object: An ee.Geometry object
    """

    geo_json = json.loads(geo_json)
    features = ee.FeatureCollection(geo_json['features'])
    return features

aoi = geojson_to_ee(gjson)

## Pre-process and Download CHIRPS datasets from GEE

In [14]:
gee_collection_id = "UCSB-CHG/CHIRPS/DAILY"
band = "precipitation"

start_date = datetime.date(2018, 4, 1)
end_date = datetime.date(2019, 12, 31)


In [21]:
precip = ee.ImageCollection(gee_collection_id).select(band)

precip = (ee.ImageCollection(gee_collection_id)
          .select(band)
          .filter(ee.Filter.date(start_date.isoformat(), end_date.isoformat()))  
         )

# Convert from daily precipitation values bin (rainy/not-rainy). 
precip_event = precip.map(lambda img: img.gt(0).copyProperties(img, img.propertyNames()))

# get sum of rainy days per month (one raster per month per year)
months = ee.List.sequence(1, 12)
years = ee.List.sequence(start_date.year, end_date.year)

by_month_year = ee.ImageCollection.fromImages(
  years.map(lambda y: months.map(lambda m:
        (precip_event
        .filter(ee.Filter.calendarRange(y, y, 'year'))
        .filter(ee.Filter.calendarRange(m, m, 'month'))
        .sum()                              
  )
).flatten()))
    
    
# Get the monthly average over all years 
by_month = ee.ImageCollection.fromImages(
  months.map(lambda m:
        (by_month_year
        .filterMetadata('month', 'equals', m)
        .mean()
        .clip(aoi)
  )))

In [23]:
## Export

def coord_list(geom):
    coords = list(geom.exterior.coords)
    return (coords)

images = by_month.toBands() # put images to band. Whell we could do a toList and create file per month but it seems to me that one file with 12 bands is a good option.

task = ee.batch.Export.image(images, 'RainyDays_CHIRPS',
  {
  'scale': 5000,
  'maxPixels': 1.0E13,
  'region': coord_list(gpd_aoi.geometry[0]),
  'fileFormat': 'GeoTIFF',
  'formatOptions': {
    'cloudOptimized': True
  },
      'folder': 'GEE_Downloads'
})

task.start()